In [6]:
import os

from osgeo import gdal
from osgeo import ogr
import numpy as np

def sample_raster(input_image, input_training_points, output_dir, class_id_field):
    """
    output a sample info file (tells how point ids map to text ids, etc.)
    points should be in same projection as the image
    """
    driver = ogr.GetDriverByName('ESRI Shapefile')
    pnt_ds = driver.Open(training_shapefile, 0) # 0 means read-only. 1 means writeable.
    layer = pnt_ds.GetLayer()
    
    im_ds = gdal.Open(input_image)
    arr = im_ds.ReadAsArray()
    geotran = image.GetGeoTransform()
    ulx = geotran[0]
    cell_width = geotran[1]
    uly = geotran[3]
    cell_height = geotran[5]
    
    classes = []
    
    training_sample_file = open(os.path.join(output_dir, os.path.basename(input_training_points)[:-4] + os.path.basename(input_image)[:-4] + ".dat"))
    header = "id,x,y"
    for b in range(len(arr)):
        header = header + "," + os.path.basename(input_image)[:-4] + "." + str(b + 1)
    header+=",response \n" # the class label
    training_sample_file.write(header)
    
    for feature in layer:
        class_type = feature.GetField(class_id_field)
        if class_type not in classes:
            classes.append(class_type)
        fid = str(float(feature.GetFID()))
        geom = feature.GetGeometryRef()
        x = geom.GetPoints()[0][0]
        y = geom.GetPoints()[0][1]
        sample_str = fid + "," + str(x) + "," + str(y) + ","
        col = (x - ulx) / cell_width
        row = abs((uly - y) / cell_height)
        sample_slice = arr[:, row, col]
        for s in sample_slice:
            sample_str += "," + str(s)
        sample_str += "," + str(float(classes.index(class_type))) + "\n"
        training_sample_file.write(sample_str)

In [7]:
driver = ogr.GetDriverByName('ESRI Shapefile')
pnt_ds = driver.Open("C:/Users/4ja/data/neighborhood_mapping/experiments/exp_1/training_points50.shp", 0) # 0 means read-only. 1 means writeable.

In [9]:
layer = pnt_ds.GetLayer()
for f in layer:
    geom = f.GetGeometryRef()
    break

In [16]:
arr = ['hi','j','lol','hg']
arr.index('j')

1